In [1]:
import re
import random
import numpy as np 
import pickle

In [2]:
def remove_space_from_quatation(string):
    temp = string.split()
    
    final_str = ''
    
    for i in temp:
        
        if i=='’':
            t = final_str[:len(final_str)-1] + "'"
            final_str = t
        else:
            final_str += i
            final_str += ' '
            
    return final_str
            
def get_tokens(statement):
    fs = remove_space_from_quatation(statement)
    count = 0
    ans = re.findall(r'[\'\w]+', fs)
    
    final_tokens = []
    
    for i in ans:
        final_tokens.append( i.lower())
        count += 1
        
    return final_tokens ,count 


def tokenize(file):
    count = 0
    f = open(file)
    lines = f.readlines()
    
    tokens = []
    
    for i in lines:
        temp ,c = get_tokens(i)
        
        count += c
        tokens.append(temp)
        
    return tokens ,count

### Adding Padding 

In [3]:
def add_padding(tokens):
    start_token = '<s>'
    end_token = '<e>'

    final_tokens = []

    for i in tokens:
        temp = [start_token] + i + [end_token]
        final_tokens.append(temp)

    return final_tokens

In [4]:
def preproceesing( filename ):
    tokens , total_tokens = tokenize(filename)
    
    padded_tokens = add_padding(tokens)
    
    return padded_tokens ,total_tokens

In [5]:
tokens, total_tokens = preproceesing('../text_data/Health_English.txt')

### Calculating grams

In [6]:
def unigram(tokens):
    grams = []
    
    for stmt in tokens:
        for i in stmt:
            temp = tuple([i])
            grams.append(temp)
    
    return grams

def bigram(tokens):
    
    gram = []
    
    for line in tokens:
        for i in range( len(line) - 1):
            temp = tuple( [line[i] ,line[i+1]])
            gram.append( temp)
            
    return gram 


def trigram(tokens):
    
    gram = []
    
    for line in tokens:
        for i in range( len(line) - 2):
            temp = tuple( [line[i] ,line[i+1],line[i+2]])
            gram.append(temp)
            
    return gram


def fourgram(tokens):
    gram = []
    
    for line in tokens:
        for i in range( len(line) - 3):
            temp = tuple( [line[i] ,line[i+1] , line[i+2] ,line[i+3] ])
            gram.append(temp)
    
    return gram


n_grams = [ None ,unigram(tokens) , bigram(tokens) , trigram(tokens) , fourgram(tokens) ]


In [7]:
def calc_freq( n_grams ):
    freq = [ None , {} ,{} ,{} ,{} ]
    
    for n in range(1,5):
        gram = n_grams[n]
        
        for i in gram:
            if i not in freq[n]:
                freq[n][i] = 1
            else:
                freq[n][i] += 1
                
    return freq

freq = calc_freq(n_grams)

In [16]:
def kn_recur( word , prev_words , top_order , freq , total_tokens):
    
    ## Base Condition
    if len(prev_words) == 0:
        if top_order == True:
            count = 0
            
            if word in freq[1]:
                count = freq[1][word]
        
            return ( count / total_tokens )
        else:
            num = 0

            for i in freq[2]:
                if i[1] == word:
                    num+=1

            return num / len( freq[2] )

    d = 0.7
    words = prev_words + [word]
    
    ## Compute NUmerator
    look_all = len(words)
    look_prev = len(prev_words)
    key = tuple(words)
    num1 = 0
    
    if top_order == True:
        ## Count For highest Order
        if key in freq[look_all]:
            num1 = max( freq[look_all][key] - d , 0 )
    
    else:
        temp = words[1:]
        
        for i in freq[look_all]:
            t1 = list(i)
            t2 = t1[ 1:len(t1)]
            t3 = tuple(t2)
            
            if t3 == temp:
                num1 += 1
    
    
    ## Computing Denominator
    key = tuple(prev_words)
    deno = 0
    
    if top_order == True:
        if key in freq[look_prev]:
            deno = freq[look_prev][key]
        else:
            deno = 1
    else:
        deno = len(freq[look_all])
    
    
    num2 = 0
    for i in freq[look_all]:
        t1 = list(i)
        t2 = t1[0:len(t1)-1]
        t3 =  tuple(t2)
        
        if t3 == tuple(prev_words):
            num2 += 1
    
    if num1==0 and num2==0:
        ans = random.uniform(0.1, 0.2) +  random.uniform(0,1) * kn_recur(word ,prev_words[1:] ,False ,freq ,total_tokens)
        return ans
    
    if num2 == 0:
        return ( num1 / deno )
    
    return (num1/deno) + d*(num2/deno)*kn_recur(word , prev_words[1:] ,False ,freq ,total_tokens)

In [11]:
def val( gram ,freq):
    ans = 0
    if len(gram) == 0 :
        for i in freq[1].values():
            ans += i
    elif gram in freq[ len(gram)].keys():
        ans = freq[len(gram)][gram]
    else:
        ans = 0
    
    return ans 

def lambdas( gram ,freq):
    
    count = 0
    
    for i in freq[ len(gram)+1 ].keys():
        if i[:-1] == gram:
            count += 1
            
    deno = val(gram ,freq)
        
    deno = deno + count
    
    if deno != 0:
        lam = count/float(deno)
        return lam
    
    return random.uniform(0.1, 0.2) 


def wb( words , freq):
    
    if len(words) == 1 :
        gram = tuple([words])
        num = val(gram ,freq)
        
        deno = 0
        for i in freq[1].values():
            deno += i
        
        return num/float(deno)
    
    gram = tuple(words)
    
    num = val( gram ,freq)
    deno = val( gram[:-1] ,freq)
    
    if deno != 0 :
        
        ans1 = ( 1-lambdas(gram[:-1] , freq) )
        ans2 =  lambdas(gram[:-1] ,freq ) * wb(gram[1:] ,freq)
        
        ans = ans1*(num/deno) + ans2
        
        return ans 
    
    else:
        ans = random.uniform(0.1, 0.2) +  (lambdas(gram[:-1] ,freq) * wb(gram[1:] ,freq))
        return ans
    

In [12]:
def find_perp( stmt ,freq ,total_tokens ,smoothing):
    
    gram4 = fourgram([stmt])

    probability = 1
    prev_prob = 1
    
    for i in gram4:
        
        cur_word = i[3]
        prev_words = list(i[0:3])
        
        if smoothing == 'kn':
            temp = kn_recur(cur_word, prev_words ,True ,freq ,total_tokens)
        else:
            temp = wb(i ,freq)
            
        if temp == 0:
            continue

        prev_prob = probability
        probability *= temp
        
        if probability == 0 :
            probability = prev_prob
            break

    deno = 1/probability
    perplexity = np.power( deno ,1/len(stmt))
    
    return perplexity

### Test 

In [13]:
f = open('../text_data/Health_English.txt')
lines = f.readlines()

In [14]:
idx = random.sample(range(0, len(tokens)), 1000)

test = []
train = [] 
stmts = []

total_tokens = 0
for i in range(0, len(tokens)):
    
    if i in idx:
        test.append( tokens[i] )
        stmts.append( lines[i] )
    else:
        train.append(tokens[i] )
        total_tokens += len(tokens[i])
        
        
n_grams = [ None ,unigram(train) , bigram(train) , trigram(train) , fourgram(train) ]

train_freq = calc_freq(n_grams)

In [17]:
health_corpus_kney = [] 

for i in range(len(test)):
    perp = find_perp(test[i] , train_freq ,total_tokens ,'kn' )
    
    print(i,test[i] ,perp)
    print()
    health_corpus_kney.append([test[i],perp])

0 ['<s>', 'you', 'may', 'keep', 'your', 'teeth', 'clean', 'and', 'breath', 'fresh', 'by', 'the', 'help', 'of', 'some', 'easy', 'tips', 'given', 'here', '<e>'] 173.93193522053122

1 ['<s>', 'exercise', 'daily', '<e>'] 1.7757347037891942

2 ['<s>', 'keep', 'doing', 'small', 'physical', 'activities', 'like', 'climbing', 'stairs', 'gardening', 'small', 'domestic', 'works', 'or', 'dancing', '<e>'] 19.04713324149883

3 ['<s>', 'these', 'retain', 'the', 'flexibility', 'in', 'the', 'body', '<e>'] 17.5872817500576

4 ['<s>', 'drink', 'plenty', 'of', 'water', 'and', 'get', 'urine', 'culture', 'done', '<e>'] 71.67045109089442

5 ['<s>', 'bones', 'become', 'weak', 'and', 'starts', 'melting', '<e>'] 30.663575540769003

6 ['<s>', 'so', 'why', 'should', 'it', 'be', 'left', 'for', 'eyes', '<e>'] 3.3517016869118588

7 ['<s>', 'but', 'later', 'these', 'habits', 'can', 'have', 'lethal', 'impact', 'specially', 'on', 'the', 'capacity', 'of', 'the', 'brain', 'to', 'remember', '<e>'] 348.1933333941813

8 ['<

52 ['<s>', 'the', 'size', 'of', 'the', 'swelling', 'decreases', 'after', 'menopause', '<e>'] 198.55405657000745

53 ['<s>', 'as', 'the', 'age', 'increases', 'the', 'cases', 'of', 'arthritis', 'also', 'increase', '<e>'] 20.699862880321138

54 ['<s>', 'if', 'there', 'is', 'any', 'wart', 'knot', 'or', 'wound', 'on', 'any', 'part', 'of', 'the', 'body', 'and', 'still', 'does', 'not', 'dry', 'up', 'quickly', 'then', 'be', 'cautious', 'this', 'can', 'be', 'skin', 'cancer', '<e>'] 335.8879991830583

55 ['<s>', 'in', 'preliminary', 'stage', 'the', 'knot', 'is', 'painless', 'and', 'because', 'of', 'this', 'reason', 'we', 'commit', 'grave', 'mistake', '<e>'] 221.09346993996988

56 ['<s>', 'at', 'times', 'lymph', 'glands', 'are', 'also', 'cut', 'and', 'removed', 'with', 'cancerous', 'skin', '<e>'] 128.95408423436768

57 ['<s>', 'if', 'pain', 'occurs', 'in', 'the', 'knot', 'of', 'skin', 'then', 'be', 'attentive', 'and', 'definitely', 'get', 'the', 'cancer', 'test', 'done', '<e>'] 445.9125555364453


102 ['<s>', 'a', 'glass', 'vessel', 'full', 'of', 'blood', 'in', 'which', 'oxygen', 'is', 'released', 'in', 'the', 'form', 'of', 'bubbles', '<e>'] 813.6023863151518

103 ['<s>', 'heart', 'lung', 'machine', 'is', 'made', 'up', 'of', 'such', 'roller', 'pumps', 'and', 'artificial', 'lungs', 'only', 'by', 'which', 'the', 'most', 'complicated', 'of', 'surgeries', 'are', 'also', 'possible', '<e>'] 128.96868250476425

104 ['<s>', 'these', 'days', 'there', 'is', 'discussion', 'about', 'ultrasonography', '<e>'] 420.33374631273455

105 ['<s>', 'the', 'properly', 'formed', 'foetus', 'may', 'also', 'get', 'destroyed', 'by', 'some', 'or', 'the', 'other', 'shortcoming', 'or', 'defect', 'of', 'the', 'placenta', '<e>'] 307.2626729244186

106 ['<s>', 'in', 'the', 'initial', 'condition', 'though', 'there', 'is', 'improvement', 'in', 'sight', 'by', 'the', 'use', 'of', 'spectacles', 'but', 'with', 'increase', 'in', 'cataract', 'one', 'feels', 'the', 'need', 'to', 'change', 'the', 'spectacles', '<e>'] 4964

146 ['<s>', 'if', 'high', 'blood', 'pressure', 'diabetes', 'have', 'you', 'surrounded', 'then', 'it', 'is', 'important', 'to', 'control', 'it', 'immediately', 'through', 'medicine', 'food', 'intake', 'and', 'exercise', '<e>'] 217.37951191651692

147 ['<s>', 'early', 'morning', 'walk', 'and', 'evening', 'walk', 'from', 'this', 'point', 'of', 'view', 'are', 'best', '<e>'] 98.70254317406571

148 ['<s>', 'but', 'its', 'not', 'even', 'like', 'it', 'can', 'be', 'considered', 'a', 'problem', 'of', 'increasing', 'age', 'because', 'due', 'to', 'weakening', 'of', 'pelvis', 'muscles', 'and', 'tissues', 'connected', 'to', 'them', 'on', 'account', 'of', 'pregnancy', 'labor', 'continuously', 'picking', 'up', 'heavy', 'things', 'menopause', 'or', 'obesity', 'the', 'disease', 'occurs', '<e>'] 2838.3149107358863

149 ['<s>', 'during', 'labor', 'during', 'the', 'birth', 'of', 'child', 'by', 'the', 'tension', 'from', 'nearby', 'muscles', 'they', 'get', 'torn', 'and', 'worn', 'by', 'which', 'the', 'danger

192 ['<s>', 'every', 'moment', 'an', 'eye', 'must', 'be', 'kept', 'on', 'the', 'development', 'of', 'the', 'child', '<e>'] 10.439101377124613

193 ['<s>', 'in', 'order', 'to', 'make', 'him', 'quiet', 'putting', 'the', 'nipple', 'repeatedly', 'on', 'his', 'mouth', '<e>'] 127.85172303906015

194 ['<s>', 'in', 'home', 'also', 'the', 'solution', 'can', 'be', 'prepared', '<e>'] 3.9784913853491584

195 ['<s>', 'it', 'is', 'better', 'to', 'feed', 'him', 'with', 'spoon', 'and', 'feeding', 'cup', 'than', 'feeding', 'with', 'the', 'bottle', '<e>'] 54.490745467751886

196 ['<s>', 'protect', 'the', 'child', 'from', 'cold', '<e>'] 1258.545727771173

197 ['<s>', 'it', 'would', 'be', 'better', 'if', 'you', 'make', 'the', 'child', 'sleep', 'in', 'the', 'mosquito', 'net', '<e>'] 3092.5863546653086

198 ['<s>', 'though', 'some', 'new', 'vaccines', 'are', 'not', 'included', 'in', 'the', 'list', 'of', 'vaccines', 'but', 'in', 'order', 'to', 'be', 'safe', 'from', 'many', 'dangerous', 'diseases', 'it', 'is'

...

808 ['<s>', 'sometimes', 'this', 'also', 'happens', 'that', 'we', 'express', 'ourselves', 'through', 'the', 'medium', 'of', 'dance', 'this', 'art', 'itself', 'is', 'dance', 'therapy', '<e>'] 32.717779586503994

809 ['<s>', 'we', 'get', 'a', 'huge', 'satisfaction', 'from', 'crying', '<e>'] 30.53604822232723

810 ['<s>', 'you', 'stress', 'and', 'problem', 'also', 'get', 'resolved', 'from', 'a', 'good', 'sleep', '<e>'] 24.12455599814217

811 ['<s>', 'painting', 'and', 'drawing', 'therapy', 'often', 'on', 'seeing', 'a', 'painting', 'and', 'drawing', 'we', 'know', 'his', 'expressions', 'and', 'personality', '<e>'] 3.5726102427744695

812 ['<s>', 'specialists', 'have', 'to', 'say', 'that', 'laughing', 'loudly', 'is', 'very', 'beneficial', 'for', 'our', 'body', '<e>'] 868.41793509199

813 ['<s>', 'research', 'and', 'conditions', 'tell', 'us', 'that', 'it', 'is', 'the', 'disease', 'of', 'mind', 'only', 'that', 'does', 'not', 'allow', 'a', 'human', 'to', 'live', 'fully', '<e>'] 7905.71651484695

857 ['<s>', 'teeth', 'are', 'clean', 'and', 'shiny', 'like', 'pearls', 'are', 'not', 'uneven', 'then', 'you', 'are', 'healthy', '<e>'] 13.225549011611838

858 ['<s>', 'of', 'mouths', 'of', 'warts', 'are', 'blue', 'yellow', 'or', 'white', 'and', 'hot', 'fine', 'blood', 'comes', 'out', 'of', 'it', 'and', 'warts', 'are', 'tiny', 'and', 'soft', 'also', 'there', 'is', 'burning', 'sensation', 'fever', 'and', 'lack', 'of', 'appetite', 'in', 'body', 'skin', 'and', 'nails', 'are', 'white', 'then', 'you', 'should', 'consider', 'it', 'to', 'be', 'pittaj', 'piles', '<e>'] 138.82946831303803

859 ['<s>', 'patient', 'should', 'also', 'use', 'laxative', 'from', 'time', 'to', 'time', 'so', 'that', 'stool', 'does', 'not', 'get', 'obstructed', 'in', 'the', 'colon', '<e>'] 8257.10620658157

860 ['<s>', 'the', 'digestive', 'system', 'of', 'baby', 'has', 'not', 'developed', 'fully', 'hence', 'you', 'need', 'to', 'give', 'him', 'a', 'little', 'food', 'at', 'fixed', 'time', 'more', 'than', 'four', 'times', '

900 ['<s>', 'with', 'this', 'no', 'harm', 'will', 'be', 'done', 'to', 'the', 'skin', 'around', 'your', 'eyes', 'and', 'eye', 'lashes', '<e>'] 581.88201607361

901 ['<s>', 'in', 'easy', 'language', 'it', "'s", 'meaning', 'is', 'that', 'while', 'living', 'they', 'pledge', 'that', 'after', 'their', 'death', 'their', 'body', 'parts', 'will', 'be', 'given', 'to', 'some', 'dying', 'patient', 'so', 'that', 'that', 'patient', 'gets', 'an', 'oppurtunity', 'to', 'live', 'a', 'new', 'life', '<e>'] 225.71453766099987

902 ['<s>', 'now', 'what', 'do', 'we', 'mean', 'by', "'", 'brain', 'dead', "'", '<e>'] 3.8654543625623257

903 ['<s>', 'after', 'death', 'only', 'eyes', 'can', 'be', 'taken', 'out', 'at', 'home', '<e>'] 7439.608130921997

904 ['<s>', 'donate', 'eyes', 'of', 'your', 'loved', 'ones', '<e>'] 2.122263064245843

905 ['<s>', 'keep', 'a', 'pillow', 'below', 'the', 'head', 'of', 'the', 'dead', '<e>'] 5854.9997256530705

906 ['<s>', 'during', '15', 'minutes', 'all', 'those', 'exercises', 'are

948 ['<s>', 'it', 'is', 'considered', 'that', 'not', 'letting', 'them', 'drink', 'fruit', 'juice', 'is', 'also', 'included', '<e>'] 482.8462485796393

949 ['<s>', 'sometimes', 'with', 'advancing', 'age', 'or', 'atherosclerosis', 'hardness', 'of', 'arteries', 'or', 'because', 'of', 'high', 'blood', 'pressure', 'some', 'part', 'of', 'the', 'aorta', 'gets', 'weak', 'and', 'that', 'starts', 'spreading', 'towards', 'the', 'outside', '<e>'] 152.12930835542397

950 ['<s>', 'which', 'has', 'been', 'divided', 'into', 'three', 'parts', 'early', 'adolescence', '12', 'to', '14', 'years', 'middle', 'adolescence', '14', 'to', '17', 'years', 'and', 'late', 'adolescence', '17', 'to', '19', 'years', 'in', 'all', 'these', 'phases', 'there', 'comes', 'major', 'physical', 'mental', 'and', 'emotional', 'changes', 'to', 'understand', 'which', 'is', 'important', 'for', 'everyone', '<e>'] 980.376112934781

951 ['<s>', 'the', 'food', 'lifestyle', 'and', 'city', 'life', 'is', 'responsible', 'for', 'this', 'to',

990 ['<s>', 'its', 'application', 'is', 'made', 'for', 'the', 'purpose', 'of', 'maintaining', 'health', 'of', 'healthy', 'person', 'and', 'prevention', 'of', 'disease', 'of', 'patient', '<e>'] 3088.3959808504546

991 ['<s>', 'it', 'is', 'an', 'effective', 'medicine', 'of', 'many', 'ailments', 'cold', 'cough', 'coryza', 'allergy', 'respiration', 'asthma', 'weakness', 'of', 'lungs', 't', 'b', 'tuberculosis', 'weakness', 'of', 'bones', 'physical', 'weakness', 'lack', 'of', 'libido', 'diabetes', 'etc', '<e>'] 230.06661646121003

992 ['<s>', 'method', 'of', 'intake', 'and', 'amount', 'take', '1', 'to', '2', 'drops', 'with', 'milk', 'morning', 'and', 'evening', '<e>'] 63.45414469087661

993 ['<s>', 'there', 'is', 'special', 'benefit', 'from', 'moti', 'pishti', 'in', 'palpitation', 'and', 'insomnia', 'disease', '<e>'] 27.053354623115833

994 ['<s>', 'there', 'is', 'immediate', 'benefit', 'from', 'mukta', 'pishti', 'in', 'lack', 'of', 'sleep', 'harshness', 'of', 'speech', 'irritability', 'rise

In [18]:
health_corpus_wb = []

for i in range(len(test)):
    perp = find_perp(test[i] , train_freq ,total_tokens ,'wb')
    
    print(test[i] ,perp)
    print()
    health_corpus_wb.append( [test[i] , perp])

['<s>', 'you', 'may', 'keep', 'your', 'teeth', 'clean', 'and', 'breath', 'fresh', 'by', 'the', 'help', 'of', 'some', 'easy', 'tips', 'given', 'here', '<e>'] 7.987043228666522

['<s>', 'exercise', 'daily', '<e>'] 1.548026464483728

['<s>', 'keep', 'doing', 'small', 'physical', 'activities', 'like', 'climbing', 'stairs', 'gardening', 'small', 'domestic', 'works', 'or', 'dancing', '<e>'] 3.7552857612105197

['<s>', 'these', 'retain', 'the', 'flexibility', 'in', 'the', 'body', '<e>'] 3.7539986017807454

['<s>', 'drink', 'plenty', 'of', 'water', 'and', 'get', 'urine', 'culture', 'done', '<e>'] 5.835207518078552

['<s>', 'bones', 'become', 'weak', 'and', 'starts', 'melting', '<e>'] 6.198590154808142

['<s>', 'so', 'why', 'should', 'it', 'be', 'left', 'for', 'eyes', '<e>'] 3.6390306124236678

['<s>', 'but', 'later', 'these', 'habits', 'can', 'have', 'lethal', 'impact', 'specially', 'on', 'the', 'capacity', 'of', 'the', 'brain', 'to', 'remember', '<e>'] 8.852081324581569

['<s>', 'a', 'thing',

['<s>', 'if', 'there', 'is', 'any', 'wart', 'knot', 'or', 'wound', 'on', 'any', 'part', 'of', 'the', 'body', 'and', 'still', 'does', 'not', 'dry', 'up', 'quickly', 'then', 'be', 'cautious', 'this', 'can', 'be', 'skin', 'cancer', '<e>'] 5.326035215260941

['<s>', 'in', 'preliminary', 'stage', 'the', 'knot', 'is', 'painless', 'and', 'because', 'of', 'this', 'reason', 'we', 'commit', 'grave', 'mistake', '<e>'] 9.515166896325063

['<s>', 'at', 'times', 'lymph', 'glands', 'are', 'also', 'cut', 'and', 'removed', 'with', 'cancerous', 'skin', '<e>'] 3.3414073002443687

['<s>', 'if', 'pain', 'occurs', 'in', 'the', 'knot', 'of', 'skin', 'then', 'be', 'attentive', 'and', 'definitely', 'get', 'the', 'cancer', 'test', 'done', '<e>'] 8.89251048594287

['<s>', 'rice', 'water', 'means', 'the', 'dense', 'white', 'water', 'while', 'cooking', 'rice', '<e>'] 3.9965073745654314

['<s>', 'if', 'the', 'intoxication', 'due', 'to', 'bhang', 'is', 'more', 'then', 'the', 'intoxication', 'vanishes', 'by', 'having

['<s>', 'these', 'days', 'there', 'is', 'discussion', 'about', 'ultrasonography', '<e>'] 2.677837902443598

['<s>', 'the', 'properly', 'formed', 'foetus', 'may', 'also', 'get', 'destroyed', 'by', 'some', 'or', 'the', 'other', 'shortcoming', 'or', 'defect', 'of', 'the', 'placenta', '<e>'] 6.808950415028432

['<s>', 'in', 'the', 'initial', 'condition', 'though', 'there', 'is', 'improvement', 'in', 'sight', 'by', 'the', 'use', 'of', 'spectacles', 'but', 'with', 'increase', 'in', 'cataract', 'one', 'feels', 'the', 'need', 'to', 'change', 'the', 'spectacles', '<e>'] 23.55878208009717

['<s>', 'presently', 'this', 'facility', 'is', 'available', 'in', '30', '35', 'cities', 'in', 'our', 'country', '<e>'] 5.7940496534810695

['<s>', 'though', 'maximum', 'women', 'try', 'to', 'keep', 'the', 'problem', 'as', 'secret', 'but', 'the', 'treatment', 'of', 'this', 'problem', 'is', 'possible', '<e>'] 9.44341568213263

['<s>', 'these', 'patients', 'get', 'relief', 'in', '80', 'by', 'using', 'the', 'instr

['<s>', 'during', 'labor', 'during', 'the', 'birth', 'of', 'child', 'by', 'the', 'tension', 'from', 'nearby', 'muscles', 'they', 'get', 'torn', 'and', 'worn', 'by', 'which', 'the', 'danger', 'of', 's', 'u', 'i', 'increases', '<e>'] 5.294595159077537

['<s>', 'it', 'takes', 'even', 'less', 'time', 'than', '30', 'minutes', 'in', 'finishing', 'the', 'safe', 'procedure', 'of', 't', 'v', 't', '<e>'] 4.455567972160225

['<s>', 'anoint', 'the', 'boils', 'and', 'rashes', 'after', 'grinding', 'the', 'bark', 'of', 'neem', '<e>'] 5.156287755775429

['<s>', 'nowadays', 'just', 'as', 'growth', 'of', 'this', 'disease', 'is', 'happening', 'likewise', 'horrible', 'forms', 'of', 'this', 'disease', 'are', 'also', 'coming', 'forward', 'each', 'day', '<e>'] 7.631038719934709

['<s>', 'on', 'there', 'being', 'bleeding', 'from', 'nose', 'by', 'splattering', 'a', 'few', 'drops', 'of', 'cold', 'water', 'bleeding', 'stops', '<e>'] 5.7152015601905894

['<s>', 'after', 'roasting', 'ginger', 'and', 'by', 'eating'

['<s>', 'by', 'consuming', 'it', 'twice', 'daily', 'one', 'can', 'get', 'high', 'benefits', '<e>'] 9.291785175921934

['<s>', 'boil', 'little', 'bit', 'of', 'ginger', 'in', 'a', 'glassful', 'of', 'water', 'till', 'the', 'time', 'it', 'becomes', 'half', '<e>'] 6.156323188911797

['<s>', 'now', 'sieve', 'the', 'water', '<e>'] 2.538242693415526

['<s>', 'make', 'the', 'child', 'drink', 'this', 'water', 'two', 'spoonfuls', 'twice', 'daily', '<e>'] 6.171210250943137

['<s>', 'wash', 'the', 'head', 'of', 'the', 'child', 'after', 'massaging', 'the', 'paste', 'of', 'bitter', 'almond', 'on', 'the', 'scalp', 'of', 'the', 'head', '<e>'] 12.86785180017773

['<s>', 'do', 'not', 'give', 'sweet', 'food', 'more', 'than', 'required', '<e>'] 3.096839999296915

['<s>', 'the', 'children', 'may', 'be', 'given', 'sweets', 'or', 'desserts', 'at', 'the', 'interval', 'of', 'two', 'or', 'four', 'days', 'according', 'to', 'tier', 'taste', '<e>'] 9.226224011728352

['<s>', 'whatever', 'is', 'easily', 'digestible'

...

['<s>', 'quantity', 'of', 'water', 'in', 'dry', 'grapes', 'or', 'raisins', 'is', '15', '<e>'] 7.868418603864028

['<s>', 'you', 'start', 'eating', 'in', 'more', 'quantity', 'even', 'though', 'you', 'are', 'not', 'that', 'hungry', '<e>'] 10.621766905520412

['<s>', 'when', 'emotional', 'eating', 'and', 'of', 'which', 'mood', 'definitely', 'write', 'this', '<e>'] 3.75610503226324

['<s>', 'people', 'start', 'staying', 'hungry', 'for', 'weight', 'loss', 'and', 'its', 'effect', 'shows', 'clearly', 'on', 'their', 'health', '<e>'] 3.8341927901046287

['<s>', 'infection', 'spreads', 'rapidly', 'in', 'winters', 'hence', 'it', 'is', 'important', 'that', 'you', 'do', 'not', 'spend', 'too', 'much', 'time', 'close', 'to', 'sick', 'people', '<e>'] 6.195875334274308

['<s>', 'benefits', 'of', 'aerobics', 'one', 'does', 'aerobic', 'activities', 'from', '20', 'minutes', 'to', 'one', 'hour', '<e>'] 3.9967832734777335

['<s>', 'cauliflower', 'of', 'high', 'fiber', 'and', 'low', 'calorie', 'is', 'a', 'fa

['<s>', 'dryness', 'occurrring', 'on', 'head', 'may', 'look', 'like', 'dandryff', '<e>'] 2.784650713334338

['<s>', 'do', 'you', 'get', 'continuous', 'attacks', 'of', 'sneezing', 'during', 'weather', 'change', '<e>'] 3.479807486553095

['<s>', 'other', 'than', 'this', 'grinding', 'rose', 'petals', 'with', 'milk', 'everday', 'apply', 'on', 'lips', 'while', 'sleeping', '<e>'] 3.253442827775435

['<s>', 'do', 'not', 'remove', 'black', 'heads', 'with', 'a', 'pin', 'or', 'by', 'pressing', '<e>'] 3.6883332548481373

['<s>', 'like', 'complaint', 'o', 'fblood', 'deficiency', 'anemia', 'after', 'pregnancy', 'on', 'aging', 'spots', 'appear', 'by', 'staying', 'in', 'sun', 'for', 'a', 'long', 'time', '<e>'] 4.5769869220512005

['<s>', 'consumption', 'of', 'carrot', 'is', 'also', 'beneficial', 'in', 'colon', 'disease', '<e>'] 14.345562977911921

['<s>', 'it', 'is', 'called', 'blindness', 'because', 'of', 'diet', '<e>'] 8.312539007682174

['<s>', 'it', 'is', 'found', 'commonly', 'in', 'india', 'espe

['<s>', 'among', 'the', 'reasons', 'for', 'snores', 'are', 'included', 'defect', 'in', 'tonsil', 'or', 'behind', 'neck', 'and', 'adenides', 'of', 'throat', 'exccesive', 'activity', 'of', 'pituatiry', 'or', 'reduced', 'activity', 'of', 'throid', 'gland', 'deviated', 'nasal', 'septum', 'tumor', 'inside', 'nose', 'increase', 'in', 'tonge', 'etc', '<e>'] 5.70013917226185

['<s>', 'not', 'just', 'this', 'deficiency', 'of', 'iron', 'element', 'also', 'goes', 'away', 'and', 'bone', 'disease', 'can', 'also', 'be', 'cured', 'away', '<e>'] 18.083389379263377

['<s>', 'lack', 'of', 'blood', 'digestion', 'related', 'problems', 'or', 'physical', 'and', 'mental', 'weakness', 'is', 'there', 'then', 'in', 'all', 'consumption', 'of', 'carrot', 'is', 'useful', '<e>'] 9.721714628321697

['<s>', 'in', 'ayurveda', 'some', 'fruitys', 'and', 'vegetables', 'has', 'been', 'told', 'as', 'the', 'ones', 'with', 'medicinal', 'benefits', '<e>'] 3.6827082896055185

['<s>', 'after', 'operation', 'in', 'a', 'few', 'mo

['<s>', 'haemophilia', "'", 'a', "'", 'is', 'a', 'disease', 'obtained', 'from', 'kulkram', 'in', 'which', 'there', 'is', 'a', 'deficiency', 'of', 'factor', 'v', '111', 'which', 'is', 'an', 'important', 'protein', 'for', 'stopping', 'the', 'flowing', 'of', 'common', 'blood', '<e>'] 8.871012456001852

['<s>', 'is', 'it', 'safe', 'to', 'donate', 'plasma', '<e>'] 1.7195746497360433

['<s>', 'saif', 'ali', 'khan', 'had', 'to', 'be', 'admitted', 'to', 'the', 'hospital', 'some', 'time', 'back', 'owing', 'to', 'a', 'problem', 'related', 'to', 'heart', 'then', 'the', 'severe', 'heart', 'attack', 'of', 'fashion', 'designer', 'rohit', 'bal', '<e>'] 5.662530315752186

['<s>', 'in', 'depression', 'the', 'person', 'feels', 'like', 'eating', 'again', 'and', 'again', 'because', 'of', 'which', 'excess', 'fat', 'starts', 'depositing', 'in', 'the', 'body', 'and', 'the', 'danger', 'of', 'heart', 'attack', 'increases', '<e>'] 8.417410236741954

['<s>', 'your', 'family', 'history', 'must', 'also', 'be', 'ch

['<s>', 'the', 'bone', 'specialist', 'of', 'the', 'saharanpur', 'district', 'hospital', 'dr', 'ashok', 'aggarwal', 'informs', 'that', 'due', 'to', 'wearing', 'high', 'heel', 'sandels', 'and', 'slippers', 'the', 'balance', 'of', 'the', 'backbone', 'gets', 'disturbed', 'because', 'of', 'which', 'pressure', 'in', 'the', 'waist', 'nerves', 'increases', 'and', 'the', 'females', 'get', 'backache', 'problems', '<e>'] 9.168971214053531

['<s>', 'this', 'pain', 'is', 'a', 'warning', 'signal', 'hence', 'the', 'docter', 'should', 'be', 'contacted', 'immediately', 'when', 'such', 'pain', 'erupts', '<e>'] 5.395612855859886

['<s>', 'do', 'you', 'have', 'problems', 'getting', 'up', 'from', 'the', 'bed', 'in', 'the', 'morning', 'or', 'you', 'feel', 'like', 'taking', 'a', 'nap', 'for', 'a', 'while', 'keeping', 'the', 'head', 'on', 'the', 'desk', 'after', 'lunch', '<e>'] 9.027178588957097

['<s>', 'due', 'to', 'busy', 'schedule', 'and', 'lack', 'of', 'sleep', 'one', 'gets', 'tired', '<e>'] 2.9643879212

In [19]:
with open('health_corpus_kn.pkl', 'wb') as f:
    pickle.dump(health_corpus_kney, f)


In [20]:
with open('health_corpus_wb.pkl', 'wb') as f:
    pickle.dump(health_corpus_wb, f)
